In [1]:
!pip install -qq pyannote.audio==3.1.1
!pip install -qq ipython==7.34.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00

In [2]:
!pip install pyannote.database

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from huggingface_hub import login
login(token="Your-Token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import shutil
import torch
import pyannote
import numpy as np
import pytorch_lightning as pl
from pyannote.core import notebook, Segment
from pyannote.database import registry, FileFinder
from pyannote.audio import Model
from pyannote.audio import Inference
from pyannote.audio.tasks import SpeakerDiarization
from pyannote.audio.utils.metric import DiscreteDiarizationErrorRate
from copy import deepcopy
from pyannote.audio.utils.signal import binarize
from pyannote.audio.pipelines.utils import get_devices
from pyannote.core import SlidingWindowFeature, Annotation
from pyannote.audio.tasks import SpeakerEmbedding

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/pyannote/audio/tasks/segmentation/mixins.py:37: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting 

In [4]:
!PYANNOTE_DATABASE_CONFIG="/content/drive/MyDrive/Pyannote2/Data/database.yml" pyannote-database info MyDatabase.SpeakerDiarization.MyProtocol

/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)
train
   48 files
   21h56m annotated
   21h56m of speech (100%)
   206 speakers
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will b

In [5]:
from pyannote.database import registry, FileFinder

registry.load_database("/content/drive/MyDrive/Pyannote2/Data/database.yml")
dataset = registry.get_protocol("MyDatabase.SpeakerDiarization.MyProtocol", {"audio": FileFinder()})

In [6]:
from pyannote.audio import Pipeline
import torch


pretrained_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True).to(torch.device("cuda"))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


...

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [7]:
from pyannote.audio import Model
model = Model.from_pretrained("pyannote/segmentation-3.0").to(torch.device("cuda"))


pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

In [8]:
from pyannote.audio.tasks import Segmentation
task = Segmentation(
    dataset,
    duration=model.specifications.duration,
    max_num_speakers=len(model.specifications.classes),
    batch_size=64,
    num_workers=4,
    loss="bce",
    vad_loss="bce")
model.task = task
model.prepare_data()
model.setup()

/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/u

Protocol MyDatabase.SpeakerDiarization.MyProtocol does not precompute the output of torchaudio.info(): adding a 'torchaudio.info' preprocessor for you to speed up dataloaders. See pyannote.database documentation on how to do that yourself.


/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)
/usr/local/lib/python3.10/dist-packages/pyannote/audio/tasks/segmentation/speaker_diarization.py:165: UserWarning: `max_num_speakers` has been deprecated in favor of `max_speakers_per_chunk`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyannote/audio/tasks/segmentation/speaker_diarization.py:169: UserWarning: `loss` has been deprecated and has no effect.
  warnings.warn("`loss` has been deprecated and has no effect.")
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packag

In [10]:
from types import MethodType
from torch.optim import Adam
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    RichProgressBar,
)

# we use Adam optimizer with 1e-4 learning rate
def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-4)

model.configure_optimizers = MethodType(configure_optimizers, model)

# we monitor diarization error rate on the validation set
# and use to keep the best checkpoint and stop early
monitor, direction = task.val_monitor
checkpoint = ModelCheckpoint(
    monitor=monitor,
    mode=direction,
    save_top_k=1,
    every_n_epochs=1,
    save_last=False,
    save_weights_only=False,
    filename="{epoch}",
    verbose=False,
)
early_stopping = EarlyStopping(
    monitor=monitor,
    mode=direction,
    min_delta=0.0,
    patience=2,
    strict=True,
    verbose=False,
)

callbacks = [RichProgressBar(), checkpoint, early_stopping]

# we train for at most 20 epochs (might be shorter in case of early stopping)
from pytorch_lightning import Trainer
trainer = Trainer(accelerator="gpu",
                  callbacks=callbacks,
                  max_epochs=20,
                  gradient_clip_val=0.5)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Name              ┃ Type             ┃ Params ┃ Mode  ┃       In sizes ┃                          Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0 │ sincnet           │ SincNet          │ 42.6 K │ train │ [1, 1, 160000] │                       [1, 60, 589] │
│ 1 │ lstm              │ LSTM             │  1.4 M │ train │   [1, 589, 60] │  [[1, 589, 256], [[8, 1, 128], [8, │
│   │                   │                  │        │       │                │                          1, 128]]] │
│ 2 │ linear            │ ModuleList       │ 49.4 K │ train │              ? │                                  ? │
│ 3 │ classifier        │ Linear           │    387 │ train │  [1, 589, 128] │                        [1, 589, 3] │
│ 4 │ activation        │ Sigmoid          │      0 │ train │    [1, 589, 3] │                        [1, 589, 3] │
│ 5 │ validation_metric │ MetricCollection │      0 │ train │              ? │                                  ? │
└───┴───────────────────┴──────────────────┴────────┴───────┴────────────────┴────────────────────────────────────┘

Trainable params: 1.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 5                                                                          
Modules in train mode: 27                                                                                          
Modules in eval mode: 0

Output()

In [13]:
# save path to the best checkpoint for later use
finetuned_model = checkpoint.best_model_path
print(f"The finetuned model is saved to {finetuned_model}")

# Define destination path
destination_path = "/content/drive/MyDrive/Pyannote2/models/completed/finetuned_best_model.ckpt"

# Copy the best checkpoint to the destination
import shutil
shutil.copy(finetuned_model, destination_path)
print(f"Best model copied to: {destination_path}")

# Uncomment to download the checkpoint
from google.colab import files
files.download(finetuned_model)


The finetuned model is saved to /content/lightning_logs/version_1/checkpoints/epoch=2.ckpt
Best model copied to: /content/drive/MyDrive/Pyannote2/models/completed/finetuned_best_model.ckpt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
pretrained_hyperparameters = pretrained_pipeline.parameters(instantiated=True)
pretrained_hyperparameters

{'segmentation': {'min_duration_off': 0.5817029604921046,
  'threshold': 0.4442333667381752},
 'clustering': {'method': 'centroid',
  'min_cluster_size': 15,
  'threshold': 0.7153814381597874}}

In [15]:
# this takes approximately 5min to run on Google Colab GPU
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.pipeline import Optimizer

pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    clustering="OracleClustering",
).to(torch.device("cuda"))
# as reported in the technical report, min_duration_off can safely be set to 0.0
pipeline.freeze({"segmentation": {"min_duration_off": 0.0}})

optimizer = Optimizer(pipeline)
dev_set = list(dataset.development())

iterations = optimizer.tune_iter(dev_set, show_progress=False)
best_loss = 1.0
for i, iteration in enumerate(iterations):
    print(f"Best segmentation threshold so far: {iteration['params']['segmentation']['threshold']}")
    if i > 20: break   # 50 iterations should give slightly better results

/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)
/usr/local/lib/python3.10/dist-packages/pyannote/pipeline/parameter.py:160: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. U

Best segmentation threshold so far: 0.511256902440078
Best segmentation threshold so far: 0.511256902440078
Best segmentation threshold so far: 0.511256902440078
Best segmentation threshold so far: 0.511256902440078
Best segmentation threshold so far: 0.5195261032208783
Best segmentation threshold so far: 0.5195261032208783
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentation threshold so far: 0.5938270194243377
Best segmentat

In [16]:
best_segmentation_threshold = optimizer.best_params["segmentation"]["threshold"]

In [17]:
# this takes approximately 5min to run on Google Colab GPU
pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    embedding=pretrained_pipeline.embedding,
    embedding_exclude_overlap=pretrained_pipeline.embedding_exclude_overlap,
    clustering=pretrained_pipeline.klustering,
).to(torch.device("cuda"))

pipeline.freeze({
    "segmentation": {
        "threshold": best_segmentation_threshold,
        "min_duration_off": 0.0,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
    },
})

optimizer = Optimizer(pipeline)
iterations = optimizer.tune_iter(dev_set, show_progress=False)
best_loss = 1.0
for i, iteration in enumerate(iterations):
    print(f"Best clustering threshold so far: {iteration['params']['clustering']['threshold']}")
    if i > 20: break  # 50 iterations should give slightly better results

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.3304840494651402
Best clustering threshold so far: 0.4745180280621921
Best clustering threshold so far: 0.5067168591553524
Best clustering threshold so far: 0.5067168591553524
Best clustering threshold so far: 0.6364913888283514
Best clustering threshold so far: 0.6305699952626296
Best clustering threshold so far: 0.6305699952626296
Best clustering threshold so far: 0.7459238410394678
Best clustering threshold so far: 0.7459238410394678
Best clustering threshold so far: 0.7459238410

In [18]:
best_clustering_threshold = optimizer.best_params['clustering']['threshold']

In [22]:
# this takes approximately 2min to run on Google Colab GPU
from pyannote.metrics.diarization import DiarizationErrorRate
metric = DiarizationErrorRate()

finetuned_pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    embedding=pretrained_pipeline.embedding,
    embedding_exclude_overlap=pretrained_pipeline.embedding_exclude_overlap,
    clustering=pretrained_pipeline.klustering,
).to(torch.device("cuda"))

finetuned_pipeline.instantiate({
    "segmentation": {
        "threshold": best_segmentation_threshold,
        "min_duration_off": 0.0,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
        "threshold": best_clustering_threshold,
    },
})

metric = DiarizationErrorRate()

for file in dataset.test():
    # apply finetuned pipeline
    file["finetuned pipeline"] = finetuned_pipeline(file)

    # evaluate its performance
    metric(file["annotation"], file["finetuned pipeline"], uem=file["annotated"])

print(f"The finetuned pipeline reaches a Diarization Error Rate (DER) of {100 * abs(metric):.1f}% on {dataset.name} test set.")

/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:178: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(
/usr/local/lib/python3.10/dist-packages/pyannote/database/util.py:284: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_uem, names=names, dtype=dtype, delim_whitespace=True)


The finetuned pipeline reaches a Diarization Error Rate (DER) of 9.2% on MyDatabase.SpeakerDiarization.MyProtocol test set.


In [25]:
# 하이퍼파라미터 저장 예제
pipeline_config = {
    "segmentation": {
        "threshold": finetuned_pipeline.segmentation.threshold,
        "min_duration_off": finetuned_pipeline.segmentation.min_duration_off,
    },
    "clustering": {
        "method": finetuned_pipeline.clustering.method,
        "min_cluster_size": finetuned_pipeline.clustering.min_cluster_size,
        "threshold": finetuned_pipeline.clustering.threshold,
    }
}

import json
with open("/content/drive/MyDrive/Pyannote2/models/completed/pipeline_config.json", "w") as f:
    json.dump(pipeline_config, f)


In [27]:
with open("/content/drive/MyDrive/Pyannote2/models/completed/pipeline_config.json", "r") as f:
    config = json.load(f)

config

{'segmentation': {'threshold': 0.5638778303610631, 'min_duration_off': 0.0},
 'clustering': {'method': 'centroid',
  'min_cluster_size': 15,
  'threshold': 0.7459238410394678}}